In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [76]:
# Loading in dataset

import pandas as pd
import numpy as np

def make_row(data):
  new_vector = np.array()

  if isinstance(s, string):
    s.replace(",", '')

  return float(s)

df = pd.read_csv("/content/gdrive/My Drive/cleaned_data_combined_modified.csv")
    # Select a subset of features for the baseline model

selected_features = ["Q1: From a scale 1 to 5, how complex is it to make this food? (Where 1 is the most simple, and 5 is the most complex)",
                         "Q2: How many ingredients would you expect this food item to contain?",
                         "Q3: In what setting would you expect this food to be served? Please check all that apply",
                         "Q4: How much would you expect to pay for one serving of this food item?",
                         "Q5: What movie do you think of when thinking of this food item?",
                         "Q6: What drink would you pair with this food item?",
                         "Q7: When you think about this food item, who does it remind you of?",
                         "Q8: How much hot sauce would you add to this food item?"]
df = df[selected_features].copy()

In [77]:
# Add indicator variables to categorical variable people

people_col = "Q7: When you think about this food item, who does it remind you of?"

# Get rid of NaNs
df[people_col] = df[people_col].fillna('')

# Hardcoded categories
unique_categories = {"Friends", "Teachers", "Strangers", "Family", "Siblings"}

# Create new indicators for each unique category
for category in unique_categories:
    df[category] = df[people_col].str.contains(category, regex=False).astype(int)

# Drop the original people column
df.drop(columns=[people_col], inplace=True)

In [80]:
# Turn movies into BoW indicators

movies = "Q5: What movie do you think of when thinking of this food item?"
vocab = {}
def clean_movie(movie):
  movie = str(movie)
  for char in movie:
    if not char.isalpha() and not char.isspace():
      movie = movie.replace(char, '')
  movie = movie.lower()
  movie_as_list = movie.split()
  for word in movie_as_list:
    if word not in vocab:
      vocab[word] = len(vocab)
  return movie_as_list

df[movies] = df[movies].apply(clean_movie)

bag_of_word_indicators = pd.DataFrame([
    [1 if word in words else 0 for word in vocab]
    for words in df[movies]
], columns=vocab.keys())


df = pd.concat([df, bag_of_word_indicators], axis=1)
df.drop(columns=[movies], inplace=True)

In [81]:
df

,"Q1: From a scale 1 to 5, how complex is it to make this food? (Where 1 is the most simple, and 5 is the most complex)",Q2: How many ingredients would you expect this food item to contain?,Q3: In what setting would you expect this food to be served? Please check all that apply,Q4: How much would you expect to pay for one serving of this food item?,Q6: What drink would you pair with this food item?,Q8: How much hot sauce would you add to this food item?,Family,Strangers,Teachers,Siblings,...,tow,mater,mistakes,wasabi,topping,pistachio,ice,cream,comical,results
0,3,6,"Week day lunch,At a party,Late night snack",5,Coke,A little (mild),0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,"bread, meet","Week day lunch,At a party,Late night snack",5$ for a large piece,Coke,NaN,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,5,"Week day lunch,Week day dinner,Weekend lunch,W...",10dollar,cola,A moderate amount (medium),0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,6-7,"Week day lunch,Week day dinner,Weekend lunch,W...",$3,Soda,I will have some of this food item with my hot...,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,2,3 or more,"Week day lunch,Week day dinner,Weekend lunch,W...",$5,Soda,A little (mild),0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1639,2,3-5,"Week day dinner,Weekend dinner",5-Mar,Soup,NaN,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1640,3,4 to 5,"Week day lunch,Weekend lunch",About 12 to 15 dollars,Water,A little (mild),0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1641,2,"It has a minimum of two ingredients, and on av...","Week day lunch,Weekend lunch,At a party",10 dollars,Nothing in particular– probably just water,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1642,1,3 or a finite amount more,"Week day lunch,Weekend lunch,Late night snack",$1 or less (for a single roll),Any tea,A little (mild),0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
